### Altes Modell für den Zeitraum 2008 - 2013 kalibrieren und 2014 - 2018 validieren

In [1]:
import pandas as pd
from swbm_mini import prepro, predict_ts, model_correlation

# Daten laden
data_all = {
    'Germany': pd.read_csv("https://raw.githubusercontent.com/emorl/Earth-System-Modelling/main/data/Data_swbm_Germany_new.csv"),
    'Spain': pd.read_csv("https://raw.githubusercontent.com/emorl/Earth-System-Modelling/main/data/Data_swbm_Spain_new.csv"),
    'Sweden': pd.read_csv("https://raw.githubusercontent.com/emorl/Earth-System-Modelling/main/data/Data_swbm_Sweden_new.csv"),
}

# Zeitfilter 2008–2013
def filter_2008_2013(df):
    df = df.copy()
    df['time'] = pd.to_datetime(df['time'])
    return df[(df['time'].dt.year >= 2008) & (df['time'].dt.year <= 2013)]

# Parameterbereiche (nur die 4 alten Parameter!)
cs_vals = [210, 420, 840]
a_vals = [2, 4, 8]
g_vals = [0.2, 0.5, 0.8]
b0_vals = [0.4, 0.6, 0.8]

results_old = []

for site, df in data_all.items():

    df_filtered = filter_2008_2013(df).reset_index(drop=True)
    df_prepro = prepro(df_filtered)

    for cs in cs_vals:
        for a in a_vals:
            for g in g_vals:
                for b0 in b0_vals:

                    config = {
                        'c_s': cs,
                        'a': a,
                        'g': g,
                        'b0': b0
                    }

                    moisture, runoff, et_flux = predict_ts(df_prepro, config)

                    corrs = model_correlation(
                        df_prepro,
                        (moisture, runoff, et_flux)
                    )

                    results_old.append({
                        'site': site,
                        'c_s': cs,
                        'a': a,
                        'g': g,
                        'b0': b0,
                        **corrs
                    })

df_results_old = pd.DataFrame(results_old)
df_results_old.to_csv("old_model_results_2008_2013.csv", index=False)


/Users/emmamorlock/git/GitHub/Earth-System-Modelling/swbm_mini.py:33: RuntimeWarning: invalid value encountered in scalar power
  return b0 * (w_i / c_s) ** g


In [5]:
df_results_old = pd.read_csv("old_model_results_2008_2013.csv")

top_old = (
    df_results_old
    .sort_values(by='sum', ascending=False)
    .groupby('site')
    .head(1)
)

best_params_old = (
    top_old
    .set_index("site")[["c_s","a","g","b0"]]
    .to_dict(orient="index")
)

print(best_params_old)


{'Germany': {'c_s': 420, 'a': 8, 'g': 0.2, 'b0': 0.6}, 'Spain': {'c_s': 210, 'a': 2, 'g': 0.8, 'b0': 0.6}, 'Sweden': {'c_s': 210, 'a': 8, 'g': 0.2, 'b0': 0.6}}


In [6]:
# Zeitfilter 2014–2018
def filter_2014_2018(df):
    df = df.copy()
    df['time'] = pd.to_datetime(df['time'])
    return df[(df['time'].dt.year >= 2014) & (df['time'].dt.year <= 2018)]

results_validation_old = []

for site, df in data_all.items():

    df_filtered = filter_2014_2018(df).reset_index(drop=True)
    df_prepro = prepro(df_filtered)

    config = best_params_old[site]

    moisture, runoff, et_flux = predict_ts(df_prepro, config)

    corrs = model_correlation(
        df_prepro,
        (moisture, runoff, et_flux)
    )

    results_validation_old.append({
        'site': site,
        **config,
        **corrs
    })

df_old_2014_2018 = pd.DataFrame(results_validation_old)

print(df_old_2014_2018)
df_old_2014_2018.to_csv("old_model_results_2014_2018.csv", index=False)


      site  c_s  a    g   b0        sm        ro        et       sum
0  Germany  420  8  0.2  0.6  0.891957  0.695677  0.900872  2.488506
1    Spain  210  2  0.8  0.6  0.906046  0.819447  0.875835  2.601327
2   Sweden  210  8  0.2  0.6  0.255149 -0.017344  0.869833  1.107638


In [8]:
new_model_results = pd.read_csv("model_results_2014_2018.csv")
print(new_model_results)

      site  c_s  a    g   b0  i_max  t_ice        sm        ro        et  \
0  Germany  420  8  0.2  0.6     20     -3  0.891697  0.745805  0.900593   
1    Spain  210  4  0.8  0.6     20     -3  0.891673  0.749034  0.880175   
2   Sweden  210  8  0.2  0.6    200     -3  0.304116 -0.004571  0.869995   

        sum  
0  2.538094  
1  2.520882  
2  1.169540  


In [10]:
print(df_old_2014_2018.round(2))


      site  c_s  a    g   b0    sm    ro    et   sum
0  Germany  420  8  0.2  0.6  0.89  0.70  0.90  2.49
1    Spain  210  2  0.8  0.6  0.91  0.82  0.88  2.60
2   Sweden  210  8  0.2  0.6  0.26 -0.02  0.87  1.11


In [9]:
print(new_model_results.round(2))


      site  c_s  a    g   b0  i_max  t_ice    sm    ro    et   sum
0  Germany  420  8  0.2  0.6     20     -3  0.89  0.75  0.90  2.54
1    Spain  210  4  0.8  0.6     20     -3  0.89  0.75  0.88  2.52
2   Sweden  210  8  0.2  0.6    200     -3  0.30 -0.00  0.87  1.17
